In [40]:
# %%file strategy_3.py

import import_ipynb
from strategy_base import TradeStrategyBase

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class TradeStrategy3(TradeStrategyBase):
    """
    Trade Strategy 3: Mean Reversion.
    Overview:
    According to probability density distribution (Assuming p_change follows normal distribution). So: 
    Pr(µ - 1σ <= X <= µ + 1σ) ~= 0.6827
    Pr(µ - 2σ <= X <= µ + 2σ) ~= 0.9545
    Pr(µ - 3σ <= X <= µ + 3σ) ~= 0.9973
    This should hold true for µ moving_average_x_days and corresponding σ moving_std_x_days. 
    Therefore, we can use a __std_mutiplier to control probability distribution, and use the following strategy.
    
    Buy when price >= moving_average_x_days + __std_mutiplier * moving_std_xdays & Sell when price reverted to moving_average
    Sell when price <= moving_average_x_days - __std_mutiplier * moving_std_xdays & Buy when price reverted to moving_average
    Assumptions: 
    moving_average_x_days and moving_std_x_days follows normal distribution (To be tested)
    Requirement: 
    1. This strategy should work with any highly liquid security. Preferably Forex or Crypto.
    2. This strategy should work whether trend is up or down, volatile or not. 
    3. The more volatile, the better, as we need to quickly close the position after mean reversion, BEFORE moving_average moved too much. 
    Constants: 
    Params:
    _std_mutiplier: The multiplier for moving standard deviation.
    _x_days: The number of days to calculate moving_average & moving_std. 
    """
    # The multiplier for moving standard deviation
    _std_mutiplier = 2.0
    # The number of days to calculate moving_average & moving_std.
    _x_days = 5
    # Position Decision. E.g. LONG: 1, SHORT: -1, NONE: 0
    _position = 0
    
    def __init__(self, x_days:int = 5, std_mutiplier:float = 2.0, should_log:bool = False, should_plot:bool = False, axs = None):
        """
        Initialize Strategy with given parameters
        :type x_days: int
        :type std_mutiplier: float 
        :type should_log: bool
        :type should_plot: bool
        :param x_days: The number of days to calculate moving_average & moving_std. E.g. 5 = 5 days.
        :param std_mutiplier: The multiplier for moving standard deviation. E.g. 2.0 means µ +- 2σ 
        :param should_log Whether or not log the strategy
        :param should_plot Whether or not plot the strategy
        """
        self._std_mutiplier = std_mutiplier
        self._x_days = x_days
        self._should_log = should_log
        self._should_plot = should_plot
        self._axs = axs
        
        self._position = 0
        self._long_tradedays =  pd.DataFrame()
        self._short_tradedays = pd.DataFrame()
        
    def trade(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Perform Analysis, Decide on Trade, and Execute Trade
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        :return: 
        """
        
        """
        Validate Data, Check tradeday has column 'ma_x' & 'mov_std_x' & 'ewm_std_x'.
        """
        if not hasattr(tradeday, 'ma'):
            # Data Incomplete, do not trade
            return
            
        """
        Firstly, add tradeday to position DataFrame
        """
        if self._position == 1:
            # Add Long days to _long_tradedays
            self._long_tradedays = self._long_tradedays.append(tradeday)
        elif self._position == -1:
            # Add Short days to _short_tradedays
            self._short_tradedays = self._long_tradedays.append(tradeday)
        
        """
        Secondly, Closing
        """
        if self._evaluate_closing(date=date,
                                 tradeday=tradeday):
            self._execute_closing(date=date,
                                 tradeday=tradeday)
        
        """
        Thirdly, Buying
        """
        if self._evaluate_buying(date=date,
                                 tradeday=tradeday):
            self._execute_buying(date=date,
                                 tradeday=tradeday)
        
        """
        Finally, Selling
        """
        if self._evaluate_selling(date=date,
                                  tradeday=tradeday):
            self._execute_selling(date=date,
                                  tradeday=tradeday)
        
        
    def _evaluate_closing(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        ma = tradeday["ma"]
        mov_std = tradeday["mov_std"]
        emv_std = tradeday["ewm_std"]
        
        if self._position == -1:
            # Currently have short position
            close_threshold = ma # Close threshold is set to ma_x
            if tradeday.close >= close_threshold:
                # Price reverted to above close threshold
                return True
        elif self._position == 1:
            # Currently have long position
            close_threshold = ma # Close threshold is set to ma_x
            if tradeday.close <= close_threshold:
                # Price reverted to below close threshold
                return True
        return False
        
    def _execute_closing(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._should_log:
            print("- Closed on {} at ${}.".format(date, tradeday.close))
        self._position = 0
        
        
    def _evaluate_buying(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        ma = tradeday["ma"]
        mov_std = tradeday["mov_std"]
        emv_std = tradeday["ewm_std"]
        buy_threshold = ma - self._std_mutiplier * mov_std
        # print("Close {} > {} ({} - {} * {})".format(tradeday.close, buy_threshold, ma, self._std_mutiplier, mov_std))
        
        if tradeday.open < buy_threshold:
            # When BUY agrees
            if self._position == 0:
                # AND no current position
                return True
        return False
        
    def _execute_buying(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How BUY should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._should_log:
            print("Bought on {} at ${}.".format(date, tradeday.close))
        self._long_tradedays = self._long_tradedays.append(tradeday)
        self._position = 1

    def _evaluate_selling(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        Evaluate whether or not SELL should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        ma = tradeday["ma"]
        mov_std = tradeday["mov_std"]
        emv_std = tradeday["ewm_std"]
        sell_threshold = ma + self._std_mutiplier * mov_std
        
        if tradeday.open > sell_threshold:
            # When SELL agrees
            if self._position == 0:
                # AND no current position
                return True
        return False
            
    def _execute_selling(self, date:pd.Timestamp, tradeday: pd.Series):
        """
        How SELL should be executed
        :type date: pd.Timestamp
        :type tradeday: pd.Series
        :param date: The pandas DataFrame date
        :param tradeday: The pandas DateFrame Series representing a trade day
        """
        if self._should_log:
            print("Sold on {} at ${}.".format(date, tradeday.close))
        if self._should_plot:
            self._plot_trade(axs=self._axs,
                             buy_date=self._last_buy_date, 
                             sell_date=date)
        self._short_tradedays = self._short_tradedays.append(tradeday)
        self._position = -1

    def _plot_trade(self, axs, buy_date, sell_date):

        print("Todo: Plotting {} {}".format(buy_date, sell_date))

        # buy_tradeday = self._held_tradedays[buy_date.date()]
        # sell_tradeday = self._held_tradedays[sell_date.date()]
        # print("Bought at {}, Sold at {}".format(buy_tradeday.close, sell_tradeday.close))

        # drawer = plt if axs is None else axs
        # 
        # # Mark red for up and green for down, slicing using start and end
        # if sell_tradeday.close < buy_tradeday.close:
        #   drawer.fill_between(self._held_tradedays.index[buy_date:sell_date], 0,
        #                   self._held_tradedays['close'][buy_date:sell_date], color='red',
        #                   alpha=.38)
        #   is_win = False
        # else:
        #   drawer.fill_between(amex_df.index[buy_date:sell_date], 0,
        #                   amex_df['close'][buy_date:sell_date], color='green',
        #                   alpha=.38)
        #   is_win = True
        # 
        # return is_win
        
    @property
    def trade_profit(self):
        long_profit = 0.0
        if 'p_change' in self._long_tradedays.columns:
            profit_array = self._long_tradedays.p_change/100 + 1
            long_profit = np.product(profit_array) - 1
        short_profit = 0.0
        if 'p_change' in self._short_tradedays.columns:
            profit_array = self._short_tradedays.p_change/100 - 1
            short_profit = np.abs(np.product(profit_array)) - 1
        return long_profit + short_profit
    
    @property
    def p_change_threshold(self):
        return self._p_change_threshold
    
    @property
    def keep_stock_threshold(self):
        return self._hold_stock_threshold
   